#  Hacks: MovieChat Movie Database.
> Using python to create a database for my movie review website
- toc: true
- image: /images/python.png
- categories: []
- type: ap
- week: 26

In [2]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for movies table in sqlite.db

- Comment on these items in the class, purpose and defintion.
    - class Movie
    - db.Model inheritance
    - _init_ method
    - ```@property```, ```@<column>.setter```
    - create, read, update, delete methods

In [4]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the Movie class to manage actions in the 'movies' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) Movie represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class Movie(db.Model):
    __tablename__ = 'movies'  # table name is plural, class name is singular

    # Define the Movie schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _reason = db.Column(db.String(255), unique=False, nullable=False)

    # constructor of a Movie object, initializes the instance variables within object (self)
    def __init__(self, name, uid, reason, password="123qwerty",):
        self._name = name    # variables with self prefix become part of the object, 
        self._uid = uid
        self.set_password(password)
        self._reason = reason

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts uid from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows uid to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def password(self):
        return self._password[0:10] + "..." # because of security only show 1st characters

    # update password, this is conventional method used for setter
    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    # check password parameter against stored/encrypted password
    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
    
    @property
    def reason(self):
        return self._reason
    
    # a setter function, allows uid to be updated after initial object creation
    @reason.setter
    def reason(self, reason):
        self._reason = reason
        
    # check if uid parameter matches user id in object, return boolean
    def is_reason(self, reason):
        return self._reason == reason
    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from Movie(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Movies table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "reason": self.reason,
        }

    # CRUD update: updates movie name, password, phone
    # returns self
    def update(self, name="", uid="", password="", reason=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(password) > 0:
            self.set_password(password)
        if len(reason) > 0:
            self.name = reason
        db.session.add(self)
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    

/home/luke_angelini/anaconda3/lib/python3.9/site-packages/flask_sqlalchemy/model.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Movie, and will be replaced in the string-lookup table.
  super().__init__(name, bases, d, **kwargs)


InvalidRequestError: Table 'movies' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
    1. Create All Tables from db Object
    2. Movie Object Constructors
    3. Try / Except 


In [5]:
"""Database Creation and Testing """


# Builds working data for testing
def initMovies():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = Movie(name='Interstellar', uid='toby', password='123toby', reason='Beautiful graphics')
        u2 = Movie(name='Click', uid='niko', password='123niko', reason='Adam Sandler funny')


        movies = [u1, u2,]

        """Builds sample movie/note(s) data"""
        for movie in movies:
            try:
                '''add movie to table'''
                object = movie.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {movie.uid}, or error.")
                
initMovies()

Records exist uid toby, or error.
Records exist uid niko, or error.


## Check for given Credentials in movies table in sqlite.db
> Use of ORM Query object and custom methods to identify movie to credentials uid and password

- Comment on purpose of following
    1. Movie.query.filter_by
    2. movie.password

In [6]:
# SQLAlchemy extracts single movie from database matching user id
def find_by_uid(uid):
    with app.app_context():
        movie = Movie.query.filter_by(_uid=uid).first()
    return movie # returns movie object

# Check credentials by finding movie and verify password
def check_credentials(uid, password):
    # query email and return movie record
    movie = find_by_uid(uid)
    if movie == None:
        return False
    if (movie.is_password(password)):
        return True
    return False
        
#check_credentials("indi", "123qwerty")

## Create a new Movie in table in Sqlite.db
> Uses SQLALchemy and custom movie.create() method to add row.

- Comment on purpose of following
    1. movie.find_by_uid() and try/except
    2. movie = Movie(...)
    3. movie.dob and try/except
    4. movie.create() and try/except

In [7]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize movie time to see if uid exists
    uid = input("Enter your user id:")
    movie = find_by_uid(uid)
    try:
        print("Found\n", movie.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your film's name:")
    password = input("Enter your password")
    reason = input("Enter the reason you enjoyed the film")
    
    # Initialize Movie object before date
    movie = Movie(name=name, 
                uid=uid, 
                password=password,
                reason=reason
                )
           
    # write object to database
    with app.app_context():
        try:
            object = movie.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

Created
 {'id': 3, 'name': 'Puss and Boots the Last Wish', 'uid': 'Luke', 'reason': 'the animation'}


## Reading movies table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. Movie.query.all
    2. json_ready assignment, google List Comprehension

In [8]:

# SQLAlchemy extracts all movies from database, turns each movie into JSON
def read():
    with app.app_context():
        table = Movie.query.all()
    json_ready = [movie.read() for movie in table] # "List Comprehensions", for each movie add movie.read() to list
    return json_ready

read()

[{'id': 1,
  'name': 'Interstellar',
  'uid': 'toby',
  'reason': 'Beautiful graphics'},
 {'id': 2, 'name': 'Click', 'uid': 'niko', 'reason': 'Adam Sandler funny'},
 {'id': 3,
  'name': 'Puss and Boots the Last Wish',
  'uid': 'Luke',
  'reason': 'the animation'}]

# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Change blog to your own database.
- Add additional CRUD
    - Add Update functionality to this blog.
    - Add Delete functionality to this blog.

In [10]:
def update():
    uid = input("Enter your user id:")
    movie = find_by_uid(uid)
    try:
        print("Found\n", movie.read())
        pass
    except:
        return
    
    password = input("Enter your password:")
    if check_credentials(uid, password):
        pass
    else:
        return
    
    nname = input("Enter your name")
    nuid = input("Enter your new user id")
    try:
        print("Already exists\n", find_by_uid(nuid).read())
        return
    except:
        pass

    with app.app_context():
        try:
            object = movie.update(nname, nuid, "")
            print("updated\n", object.read())
        except:
            print("Unknown error")
    
        
update()

In [46]:
def delete():
    uid = input("Enter your user id:")
    movie = find_by_uid(uid)
    try:
        print("Found\n", movie.read())
        pass
    except:
        return
    
    password = input("Enter your password:")
    if check_credentials(uid, password):
        pass
    else:
        return

    with app.app_context():
        try:
            object = movie.delete()
            print("deleted\n", object.read())
        except:
            print("Unknown error")
    
        
delete()

Found
 {'id': 7, 'name': 'Luke', 'uid': 'Luke', 'dob': '03-15-2023', 'age': 0}


In [11]:
def crud():
    uinp = input("create, read, update, or delete?")
    if uinp.lower() == "create":
        create()
    elif uinp.lower() == "read":
        read()
    elif uinp.lower() == "update":
        update()
    elif uinp.lower() == "delete":
        delete()
    else:
        print("Input Error")
crud()

Created
 {'id': 4, 'name': 'Over The Hedge', 'uid': 'john', 'reason': 'Fun Comedy'}
